In [1]:
#!pip install xgboost
#!pip install lightgbm
#!pip install catboost

In [2]:
import pandas as pd
import pickle
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
import tqdm
import numpy as np
import ast
import json

## Вот эта табла нужна чтобы вытащить координаты

In [3]:
df = pd.read_excel('for_back.xlsx')

In [4]:
df

,lat,lon,azimuth,Административный округ,Район,Группа стоимости
0,55.573691,37.631423,273,Южный,Бирюлёво Западное,Дешево
1,55.584765,37.712454,232,Южный,Бирюлёво Восточное,Дешево
2,55.808425,37.388808,188,Северо-Западный,Кунцево,Дорого
3,55.674378,37.422364,333,Западный,Очаково-Матвеевское,Дорого
4,55.608396,37.766383,54,Подмосковье,Орехово-Борисово Южное,Ниже среднего
...,...,...,...,...,...,...
11802,55.694607,37.676856,93,Южный,Печатники,Средне
11803,55.776881,37.524088,40,Северный,Хорошёвский,Дорого
11804,55.623283,37.421858,324,Новомосковский,Подмосковье,Дешево
11805,55.826256,37.629043,114,Северо-Восточный,Останкинский,Средне


In [5]:
json_data = {
  "Административный округ": ['Центральный'],
  "Район": ['Тверской']
}

In [6]:
conditions = [{"Административный округ": ao, "Район": r} for ao in json_data["Административный округ"] for r in json_data["Район"]]

In [7]:
conditions 

[{'Административный округ': 'Центральный', 'Район': 'Тверской'}]

Нужно тут развернуть 

filtered_df = df[(df['Административный округ'] == 'Южный') & 
                 (df['Район'] == 'Бирюлёво Западное')]

In [8]:
def filter_df(df, conditions):
    filtered_df = pd.DataFrame()
    
    for condition in conditions:
        if condition["Административный округ"]:
            temp_df = df[df["Административный округ"] == condition["Административный округ"]]
            filtered_df = pd.concat([filtered_df, temp_df])
        if condition["Район"]:
            temp_df = df[df["Район"] == condition["Район"]]
            filtered_df = pd.concat([filtered_df, temp_df])
    
    return filtered_df.drop_duplicates().reset_index(drop=True)

In [9]:
filtered_df = filter_df(df, conditions)

In [10]:
filtered_df

,lat,lon,azimuth,Административный округ,Район,Группа стоимости
0,55.741072,37.652697,292,Центральный,Замоскворечье,Очень дорого
1,55.752387,37.610085,99,Центральный,Арбат,Очень дорого
2,55.755087,37.612469,216,Центральный,Тверской,Очень дорого
3,55.753638,37.558615,243,Центральный,Пресненский,Очень дорого
4,55.751496,37.551201,61,Центральный,Пресненский,Очень дорого
...,...,...,...,...,...,...
2061,55.792377,37.594973,270,Северо-Восточный,Тверской,Очень дорого
2062,55.780016,37.578953,136,Северный,Тверской,Очень дорого
2063,55.792371,37.600270,85,Северо-Восточный,Тверской,Очень дорого
2064,55.792182,37.578614,259,Северный,Тверской,Очень дорого


In [ ]:
количество билбордов 
n  = json(количество билбордов)

In [11]:
result = filtered_df.head(n)[['lat', 'lon', 'azimuth' , 'Группа стоимости']]
result = result.reset_index()
result.drop(columns = 'index' , inplace = True)
result

,lat,lon,azimuth,Группа стоимости
0,55.741072,37.652697,292,Очень дорого
1,55.752387,37.610085,99,Очень дорого
2,55.755087,37.612469,216,Очень дорого
3,55.753638,37.558615,243,Очень дорого
4,55.751496,37.551201,61,Очень дорого
5,55.748798,37.533277,358,Очень дорого
6,55.759778,37.625339,263,Очень дорого
7,55.763911,37.556915,104,Очень дорого
8,55.765100,37.536818,18,Очень дорого
9,55.724082,37.551748,133,Очень дорого


In [12]:
n = len(result)

In [13]:
x_test = {
    'gender': ['all'] * n, 
    'income': ['bc'] * n
}

In [14]:
X_test = pd.DataFrame(x_test)
X_test

,gender,income
0,all,bc
1,all,bc
2,all,bc
3,all,bc
4,all,bc
5,all,bc
6,all,bc
7,all,bc
8,all,bc
9,all,bc


In [15]:
X_test = pd.concat([result, X_test], axis=1)
X_test

,lat,lon,azimuth,Группа стоимости,gender,income
0,55.741072,37.652697,292,Очень дорого,all,bc
1,55.752387,37.610085,99,Очень дорого,all,bc
2,55.755087,37.612469,216,Очень дорого,all,bc
3,55.753638,37.558615,243,Очень дорого,all,bc
4,55.751496,37.551201,61,Очень дорого,all,bc
5,55.748798,37.533277,358,Очень дорого,all,bc
6,55.759778,37.625339,263,Очень дорого,all,bc
7,55.763911,37.556915,104,Очень дорого,all,bc
8,55.765100,37.536818,18,Очень дорого,all,bc
9,55.724082,37.551748,133,Очень дорого,all,bc


In [16]:
district = pd.read_excel('moscow_admin_division_wth_ao.xlsx')
district

,Субъект,min_longitude,min_latitude,max_longitude,max_latitude
0,Арбат,37.572060,55.743712,37.612228,55.758061
1,Басманный,37.627940,55.750749,37.713525,55.781956
2,Замоскворечье,37.615133,55.720280,37.654211,55.749153
3,Красносельский,37.628386,55.759797,37.688804,55.792981
4,Мещанский,37.612223,55.759120,37.644896,55.797140
...,...,...,...,...,...
120,Крюково,37.144003,55.947901,37.220825,55.999244
121,Матушкино,37.177540,55.986998,37.231254,56.018630
122,Савёлки,37.209283,55.966915,37.268406,56.004668
123,Силино,37.131448,55.980439,37.205714,56.021208


In [17]:
X_test['lat'] = X_test['lat'].astype(float)
X_test['lon'] = X_test['lon'].astype(float)
district['min_latitude'] = district['min_latitude'].astype(float)
district['max_latitude'] = district['max_latitude'].astype(float)
district['min_longitude'] = district['min_longitude'].astype(float)
district['max_longitude'] = district['max_longitude'].astype(float)

In [18]:
def get_district(lat, lon, data):
    for index, row in data.iterrows():
        if row['min_latitude'] <= lat <= row['max_latitude'] and row['min_longitude'] <= lon <= row['max_longitude']:
            return row['Субъект']
    return None

# Apply the function to each row in df with tqdm progress bar
#tqdm.pandas()
X_test['district'] = X_test.apply(lambda row: get_district(row['lat'], row['lon'], district), axis=1)

In [19]:
X_test

,lat,lon,azimuth,Группа стоимости,gender,income,district
0,55.741072,37.652697,292,Очень дорого,all,bc,Замоскворечье
1,55.752387,37.610085,99,Очень дорого,all,bc,Арбат
2,55.755087,37.612469,216,Очень дорого,all,bc,Тверской
3,55.753638,37.558615,243,Очень дорого,all,bc,Пресненский
4,55.751496,37.551201,61,Очень дорого,all,bc,Пресненский
5,55.748798,37.533277,358,Очень дорого,all,bc,Пресненский
6,55.759778,37.625339,263,Очень дорого,all,bc,Мещанский
7,55.763911,37.556915,104,Очень дорого,all,bc,Пресненский
8,55.765100,37.536818,18,Очень дорого,all,bc,Пресненский
9,55.724082,37.551748,133,Очень дорого,all,bc,Хамовники


In [20]:
X_test['district'] = X_test['district'].fillna('Подмосковье')
X_test

,lat,lon,azimuth,Группа стоимости,gender,income,district
0,55.741072,37.652697,292,Очень дорого,all,bc,Замоскворечье
1,55.752387,37.610085,99,Очень дорого,all,bc,Арбат
2,55.755087,37.612469,216,Очень дорого,all,bc,Тверской
3,55.753638,37.558615,243,Очень дорого,all,bc,Пресненский
4,55.751496,37.551201,61,Очень дорого,all,bc,Пресненский
5,55.748798,37.533277,358,Очень дорого,all,bc,Пресненский
6,55.759778,37.625339,263,Очень дорого,all,bc,Мещанский
7,55.763911,37.556915,104,Очень дорого,all,bc,Пресненский
8,55.765100,37.536818,18,Очень дорого,all,bc,Пресненский
9,55.724082,37.551748,133,Очень дорого,all,bc,Хамовники


In [21]:
def determine_billboard_side(azimuth):
    """
    Определить сторону билборда ("А" или "Б") на основе азимута.

    azimuth: float - азимут в градусах (0-360)
    return: str - "А" или "Б"
    """
    if 0 <= azimuth <= 90 or 270 <= azimuth <= 360:
        return "А"
    elif 90 < azimuth < 270:
        return "Б"
    else:
        return "Неизвестная сторона"

In [22]:
X_test['side'] = X_test['azimuth'].apply(determine_billboard_side)
X_test

,lat,lon,azimuth,Группа стоимости,gender,income,district,side
0,55.741072,37.652697,292,Очень дорого,all,bc,Замоскворечье,А
1,55.752387,37.610085,99,Очень дорого,all,bc,Арбат,Б
2,55.755087,37.612469,216,Очень дорого,all,bc,Тверской,Б
3,55.753638,37.558615,243,Очень дорого,all,bc,Пресненский,Б
4,55.751496,37.551201,61,Очень дорого,all,bc,Пресненский,А
5,55.748798,37.533277,358,Очень дорого,all,bc,Пресненский,А
6,55.759778,37.625339,263,Очень дорого,all,bc,Мещанский,Б
7,55.763911,37.556915,104,Очень дорого,all,bc,Пресненский,Б
8,55.765100,37.536818,18,Очень дорого,all,bc,Пресненский,А
9,55.724082,37.551748,133,Очень дорого,all,bc,Хамовники,Б


In [23]:
dens = pd.read_excel('Плотность районов.xlsx')
dens

,Район,Плотность
0,Академический,18736.02
1,Алексеевский,15196.79
2,Алтуфьевский,17664.00
3,Арбат,16838.39
4,Аэропорт,17313.10
...,...,...
141,Южное Тушино,13725.19
142,Южнопортовый,16418.54
143,Якиманка,5747.71
144,Ярославский,12247.68


In [24]:
dens.rename(columns={'Район': 'district'}, inplace=True)

In [25]:
X_test = pd.merge(X_test, dens, on='district', how='left')

In [26]:
X_test['Плотность'] = X_test['Плотность'].fillna(6000)

In [27]:
nedv = pd.read_excel('недвижимость.xlsx')

In [28]:
nedv.rename(columns={'Субъект': 'district'}, inplace=True)

In [29]:
X_test = pd.merge(X_test, nedv, on='district', how='left')

In [30]:
X_test['Стоимость недвижимости'] = X_test['Стоимость недвижимости'].fillna(200000)

In [31]:
X_test['lat'] = X_test['lat'].astype(float)
X_test['lon'] = X_test['lon'].astype(float)

# Преобразование координат в синусоиду и косинусоиду
X_test['lat_sin'] = np.sin(np.radians(X_test['lat']))
X_test['lat_cos'] = np.cos(np.radians(X_test['lat']))
X_test['lon_sin'] = np.sin(np.radians(X_test['lon']))
X_test['lon_cos'] = np.cos(np.radians(X_test['lon']))

In [32]:
X_test

,lat,lon,azimuth,Группа стоимости,gender,income,district,side,Плотность,Стоимость недвижимости,lat_sin,lat_cos,lon_sin,lon_cos
0,55.741072,37.652697,292,Очень дорого,all,bc,Замоскворечье,А,13573.38,396450,0.826502,0.562934,0.610874,0.791728
1,55.752387,37.610085,99,Очень дорого,all,bc,Арбат,Б,16838.39,507328,0.826613,0.562770,0.610285,0.792182
2,55.755087,37.612469,216,Очень дорого,all,bc,Тверской,Б,10659.83,480487,0.826640,0.562732,0.610318,0.792157
3,55.753638,37.558615,243,Очень дорого,all,bc,Пресненский,Б,10894.19,408401,0.826625,0.562752,0.609573,0.792730
4,55.751496,37.551201,61,Очень дорого,all,bc,Пресненский,А,10894.19,408401,0.826604,0.562783,0.609470,0.792809
5,55.748798,37.533277,358,Очень дорого,all,bc,Пресненский,А,10894.19,408401,0.826578,0.562822,0.609222,0.793000
6,55.759778,37.625339,263,Очень дорого,all,bc,Мещанский,Б,13212.61,396696,0.826686,0.562664,0.610495,0.792020
7,55.763911,37.556915,104,Очень дорого,all,bc,Пресненский,Б,10894.19,408401,0.826726,0.562604,0.609549,0.792748
8,55.765100,37.536818,18,Очень дорого,all,bc,Пресненский,А,10894.19,408401,0.826738,0.562587,0.609271,0.792962
9,55.724082,37.551748,133,Очень дорого,all,bc,Хамовники,Б,10790.28,463581,0.826335,0.563179,0.609478,0.792803


In [33]:
X_test.columns
data_train = X_test[['gender', 'income', 'district', 'side', 'Плотность', 'Стоимость недвижимости', 'lat_sin', 'lat_cos', 'lon_sin', 'lon_cos']]
data_train

,gender,income,district,side,Плотность,Стоимость недвижимости,lat_sin,lat_cos,lon_sin,lon_cos
0,all,bc,Замоскворечье,А,13573.38,396450,0.826502,0.562934,0.610874,0.791728
1,all,bc,Арбат,Б,16838.39,507328,0.826613,0.562770,0.610285,0.792182
2,all,bc,Тверской,Б,10659.83,480487,0.826640,0.562732,0.610318,0.792157
3,all,bc,Пресненский,Б,10894.19,408401,0.826625,0.562752,0.609573,0.792730
4,all,bc,Пресненский,А,10894.19,408401,0.826604,0.562783,0.609470,0.792809
5,all,bc,Пресненский,А,10894.19,408401,0.826578,0.562822,0.609222,0.793000
6,all,bc,Мещанский,Б,13212.61,396696,0.826686,0.562664,0.610495,0.792020
7,all,bc,Пресненский,Б,10894.19,408401,0.826726,0.562604,0.609549,0.792748
8,all,bc,Пресненский,А,10894.19,408401,0.826738,0.562587,0.609271,0.792962
9,all,bc,Хамовники,Б,10790.28,463581,0.826335,0.563179,0.609478,0.792803


In [34]:
data_train.loc[:, 'side'] = data_train['side'].map({'А': 1, 'Б': 0})
data_train

,gender,income,district,side,Плотность,Стоимость недвижимости,lat_sin,lat_cos,lon_sin,lon_cos
0,all,bc,Замоскворечье,1,13573.38,396450,0.826502,0.562934,0.610874,0.791728
1,all,bc,Арбат,0,16838.39,507328,0.826613,0.562770,0.610285,0.792182
2,all,bc,Тверской,0,10659.83,480487,0.826640,0.562732,0.610318,0.792157
3,all,bc,Пресненский,0,10894.19,408401,0.826625,0.562752,0.609573,0.792730
4,all,bc,Пресненский,1,10894.19,408401,0.826604,0.562783,0.609470,0.792809
5,all,bc,Пресненский,1,10894.19,408401,0.826578,0.562822,0.609222,0.793000
6,all,bc,Мещанский,0,13212.61,396696,0.826686,0.562664,0.610495,0.792020
7,all,bc,Пресненский,0,10894.19,408401,0.826726,0.562604,0.609549,0.792748
8,all,bc,Пресненский,1,10894.19,408401,0.826738,0.562587,0.609271,0.792962
9,all,bc,Хамовники,0,10790.28,463581,0.826335,0.563179,0.609478,0.792803


In [35]:
data_train['hash'] = '0038e05f5cae8b3e'
data_train

/var/folders/zl/wtf6lxgn593_7tswz2pzs80m0000gn/T/ipykernel_9769/1057672974.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['hash'] = '0038e05f5cae8b3e'


,gender,income,district,side,Плотность,Стоимость недвижимости,lat_sin,lat_cos,lon_sin,lon_cos,hash
0,all,bc,Замоскворечье,1,13573.38,396450,0.826502,0.562934,0.610874,0.791728,0038e05f5cae8b3e
1,all,bc,Арбат,0,16838.39,507328,0.826613,0.562770,0.610285,0.792182,0038e05f5cae8b3e
2,all,bc,Тверской,0,10659.83,480487,0.826640,0.562732,0.610318,0.792157,0038e05f5cae8b3e
3,all,bc,Пресненский,0,10894.19,408401,0.826625,0.562752,0.609573,0.792730,0038e05f5cae8b3e
4,all,bc,Пресненский,1,10894.19,408401,0.826604,0.562783,0.609470,0.792809,0038e05f5cae8b3e
5,all,bc,Пресненский,1,10894.19,408401,0.826578,0.562822,0.609222,0.793000,0038e05f5cae8b3e
6,all,bc,Мещанский,0,13212.61,396696,0.826686,0.562664,0.610495,0.792020,0038e05f5cae8b3e
7,all,bc,Пресненский,0,10894.19,408401,0.826726,0.562604,0.609549,0.792748,0038e05f5cae8b3e
8,all,bc,Пресненский,1,10894.19,408401,0.826738,0.562587,0.609271,0.792962,0038e05f5cae8b3e
9,all,bc,Хамовники,0,10790.28,463581,0.826335,0.563179,0.609478,0.792803,0038e05f5cae8b3e


In [36]:
grouped = data_train.groupby('hash').agg({
    'gender': 'first',
    'income': 'first',
    'side': 'sum',
    'Плотность': 'mean',
    'Стоимость недвижимости': 'mean',
    'lat_sin': lambda x: list(x),
    'lat_cos': lambda x: list(x),
    'lon_sin': lambda x: list(x),
    'lon_cos': lambda x: list(x),
}).reset_index()
df = pd.get_dummies(grouped, columns=['gender', 'income'])
df

,hash,side,Плотность,Стоимость недвижимости,lat_sin,lat_cos,lon_sin,lon_cos,gender_all,income_bc
0,0038e05f5cae8b3e,4,11954.544,428654.7,"[0.8265020442637712, 0.8266131931335186, 0.826...","[0.5629337179702485, 0.5627704940183061, 0.562...","[0.6108736068280419, 0.610284610608835, 0.6103...","[0.7917281329350997, 0.7921822353814951, 0.792...",True,True


In [37]:
def safe_literal_eval(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return val

df['lat_sin'] = df['lat_sin'].apply(safe_literal_eval)
df['lat_cos'] = df['lat_cos'].apply(safe_literal_eval)
df['lon_sin'] = df['lon_sin'].apply(safe_literal_eval)
df['lon_cos'] = df['lon_cos'].apply(safe_literal_eval)

# Вычисление среднего значения для каждого из списков
df['lat_sin'] = df['lat_sin'].apply(lambda x: np.mean(x) if isinstance(x, list) else x)
df['lat_cos'] = df['lat_cos'].apply(lambda x: np.mean(x) if isinstance(x, list) else x)
df['lon_sin'] = df['lon_sin'].apply(lambda x: np.mean(x) if isinstance(x, list) else x)
df['lon_cos'] = df['lon_cos'].apply(lambda x: np.mean(x) if isinstance(x, list) else x)

In [38]:
df

,hash,side,Плотность,Стоимость недвижимости,lat_sin,lat_cos,lon_sin,lon_cos,gender_all,income_bc
0,0038e05f5cae8b3e,4,11954.544,428654.7,0.826605,0.562783,0.609853,0.792514,True,True


In [39]:
bool_columns = ['gender_all', 'gender_female', 'gender_male', 'income_ab', 'income_abc', 'income_ac', 'income_bc', 'income_c']

In [40]:
for col in bool_columns:
    if col not in df.columns:
        df[col] = 0

# Заполнение существующих столбцов значениями
df['gender_all'] = 1
df['income_bc'] = 1

In [41]:
df

,hash,side,Плотность,Стоимость недвижимости,lat_sin,lat_cos,lon_sin,lon_cos,gender_all,income_bc,gender_female,gender_male,income_ab,income_abc,income_ac,income_c
0,0038e05f5cae8b3e,4,11954.544,428654.7,0.826605,0.562783,0.609853,0.792514,1,1,0,0,0,0,0,0


In [42]:
df = df.drop(columns=['hash'])

In [43]:
df

,side,Плотность,Стоимость недвижимости,lat_sin,lat_cos,lon_sin,lon_cos,gender_all,income_bc,gender_female,gender_male,income_ab,income_abc,income_ac,income_c
0,4,11954.544,428654.7,0.826605,0.562783,0.609853,0.792514,1,1,0,0,0,0,0,0


In [44]:
df['side'] = df['side'].astype('int64')

In [45]:
df.dtypes

side                        int64
Плотность                 float64
Стоимость недвижимости    float64
lat_sin                   float64
lat_cos                   float64
lon_sin                   float64
lon_cos                   float64
gender_all                  int64
income_bc                   int64
gender_female               int64
gender_male                 int64
income_ab                   int64
income_abc                  int64
income_ac                   int64
income_c                    int64
dtype: object

In [46]:
correct_order = ['side', 'Плотность', 'Стоимость недвижимости', 'lat_sin', 'lat_cos', 'lon_sin', 'lon_cos',
                 'gender_all', 'gender_female', 'gender_male', 'income_ab', 'income_abc', 'income_ac', 'income_bc', 'income_c']

In [47]:
df = df[correct_order]
df

,side,Плотность,Стоимость недвижимости,lat_sin,lat_cos,lon_sin,lon_cos,gender_all,gender_female,gender_male,income_ab,income_abc,income_ac,income_bc,income_c
0,4,11954.544,428654.7,0.826605,0.562783,0.609853,0.792514,1,0,0,0,0,0,1,0


In [48]:
with open('ensemble_models.pkl', 'rb') as file:
    model = pickle.load(file)
print(model)

{'xgb_model': XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...), 'catboost_model': <catboost.core.CatBoostRegressor object at 0x1442bcd10>, 'lgb_model': LGBMRegressor(colsample_bytree=0.8, learning_rate=0.01, metric='rmse',
              n_estimators=1000, objective='regression', subsample=0.8), 'rf_mode

/opt/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:57:50] WARNING: /Users/runner/work/xgboost/xgboost/src/gbm/../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


In [49]:
model

{'xgb_model': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='rmse', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...),
 'catboost_model': <catboost.core.CatBoostRegressor at 0x1442bcd10>,
 'lgb_model': LGBMRegressor(colsample_bytree=0.8, learning_rate=0.01, metric='rmse',
               n_estimators=1000, objective='regression', subsample=0.8),
 '

In [50]:
xgb_model = model['xgb_model']
catboost_model = model['catboost_model']
lgb_model = model['lgb_model']
rf_model = model['rf_model']
weights = model['weights']

In [51]:
xgb_test_pred = xgb_model.predict(df)
catboost_test_pred = catboost_model.predict(df)
lgb_test_pred = lgb_model.predict(df)
rf_test_pred = rf_model.predict(df)

In [52]:
ensemble_test_pred = (weights[0] * xgb_test_pred +
                      weights[1] * catboost_test_pred +
                      weights[2] * lgb_test_pred )

In [53]:
ensemble_test_pred

array([12.14374817])

In [54]:
result

,lat,lon,azimuth,Группа стоимости
0,55.741072,37.652697,292,Очень дорого
1,55.752387,37.610085,99,Очень дорого
2,55.755087,37.612469,216,Очень дорого
3,55.753638,37.558615,243,Очень дорого
4,55.751496,37.551201,61,Очень дорого
5,55.748798,37.533277,358,Очень дорого
6,55.759778,37.625339,263,Очень дорого
7,55.763911,37.556915,104,Очень дорого
8,55.765100,37.536818,18,Очень дорого
9,55.724082,37.551748,133,Очень дорого
